### Pre-Configuring Database Connection

Update the following variables with your actual database credentials: username, password, host, port, and service name to establish a connection.

In [ ]:
import oracledb
username = 'your_username'
# IMPORTANT: do not put passwords into notebooks in plaintext.
# Instead, fetch it from a secure password store or if given by the user,
# use the password widget: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#password
password = 'your_password'
host = 'your_host'
port = 'your_port'
service_name = 'your_service_name'

dsn = f'{host}:{port}/{service_name}'
connection = oracledb.connect(user=username, password=password, dsn=dsn)
print("Connection established successfully.")

Please check this link for data, table creation https://github.com/oracle-samples/oracle-graph/tree/master/23c-graph-demos.

### SqlGraphVisualization

The following code block tests SqlGraphVisualization features provided by the Jupyter extension.

In [ ]:
from oraclegraph import GraphVisualization, SqlGraphVisualization

query = '''
    SELECT *
    FROM GRAPH_TABLE(
        BANK_GRAPH
        MATCH (a)-[e]-(b)
        COLUMNS (
            vertex_id(a) AS aid,
            edge_id(e) AS eid,
            vertex_id(b) AS bid
        )
    )
    ORDER BY aid, eid, bid
'''

vq = SqlGraphVisualization(connection)
graph_query = vq.visual_query(query)
defaults_feature = {
    'interactionActive': True,
    'stickyActive': True
}
rule_based_styles = [
    {
        "component": "vertex",
        "target": "vertex",
        "conditions": {
            "conditions": [
                {
                    "property": "NAME",
                    "operator": "!=",
                    "value": ""
                }
            ]
        },
        "modifierStyles": {
            "hover": {
                "label": {
                    "text": "${properties.NAME}"
                }
            }
        },
        "legendTitle": "Hover - Name Caption"
    },
    {
        "component": "vertex",
        "target": "vertex",
        "conditions": {
            "conditions": [
                {
                    "property": "BALANCE",
                    "operator": ">=",
                    "value": 150000
                }
            ]
        },
        "style": {
            "color": "#AA5F71"
        },
        "legendTitle": "Balance >= $150,000"
    },
    {
        "component": "vertex",
        "target": "outgoing",
        "conditions": {
            "conditions": [
                {
                    "property": "BALANCE",
                    "operator": ">=",
                    "value": 150000
                }
            ]
        },
        "animations": [
            [
                {
                    "id": "animation_1",  # Corrected typo from "animiation_1" for clarity
                    "duration": 5,
                    "timingFunction": "linear",
                    "keyFrames": [
                        {
                            "percentage": 100,
                            "style": {
                                "strokeDashoffset": 100
                            }
                        }
                    ]
                }
            ]
        ],
        "style": {
            "color": "#825BBD",
            "dasharray": "dashed3"
        },
        "legendTitle": "Edge Animation"
    }
]
settings = {
    'numberOfHops': 2,
    'defaults': defaults_feature,
    'ruleBasedStyles': rule_based_styles
}
 

bank_graph = GraphVisualization(data=graph_query, settings=settings)
bank_graph.height=600
display(bank_graph)

In [ ]:
# Close the graph query to free up resources once done
bank_graph.close()

In [ ]:
# Re-execute the query using the same SqlGraphVisualization instance, or run a different query
graph_query = vq.visual_query(query)
bank_graph = GraphVisualization(data=graph_query, settings=settings)
display(bank_graph)

### Closing the SqlGraphVisualization Instance

Please close the SqlGraphVisualization instance to release database resources.

In [ ]:
# Close all graph queries with this SqlGraphVisualization instance
vq.close()

# Explicitly close the database connection
connection.close()